### **Imports**

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import SGD
from keras.metrics import binary_accuracy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Create model**

In [ ]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

In [ ]:
# make model
model = define_model()
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


### **Train model**

In [ ]:
# prepare iterators
train_it = datagen.flow_from_directory("./drive/Shareddrives/EECS 351 Project/cats_and_dogs_filtered/train" , class_mode='binary', shuffle=True, batch_size=64, target_size=(200, 200))
test_it = datagen.flow_from_directory("./drive/Shareddrives/EECS 351 Project/cats_and_dogs_filtered/validation", class_mode='binary', shuffle=False, batch_size=64, target_size=(200, 200))

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# fit model
history = model.fit(train_it, steps_per_epoch=len(train_it),
validation_data = test_it, validation_steps=len(test_it), epochs=20, verbose=0)

### **Test model**

In [ ]:
# evaluate model and print accuracy & loss
loss, accuracy = model.evaluate(test_it, steps=len(test_it), verbose=0)
print('Test loss: > %.3f' % (loss));
print('Test accuracy: > %.3f' % (accuracy * 100.0))

### **Create Graphs**

In [ ]:
# make confusion matrix
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix

##predict based on test data
predicted_labels = model.predict(test_it);

In [ ]:
##generate correct labels 
CorrectLabels = []
for i in range(0,2500):
    CorrectLabels.append(0);
for i in range(0,2500):
    CorrectLabels.append(1);

##turn probabilities to binary values
actualPredictedLabels = [];

for i in range(0,5000):
  if predicted_labels[i] > 0.5:
   actualPredictedLabels.append(1);
  else:
    actualPredictedLabels.append(0);

cm = confusion_matrix(CorrectLabels, actualPredictedLabels)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

In [ ]:
##plot losses & accuracy over epochs
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()